### Use base conda env

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
def plot_training_metrics(csv_file_path: str, output_dir: str = 'training_plots'):
    if not os.path.exists(csv_file_path):
        print(f"Greška: Fajl '{csv_file_path}' ne postoji.")
        return

    try:
        df = pd.read_csv(csv_file_path)
    except Exception as e:
        print(f"Greška pri čitanju CSV fajla '{csv_file_path}': {e}")
        return

    expected_metrics = [
        'loss', 'accuracy', 'f1_score', 'roc_auc', 'average_precision'
    ]
    
    if 'lr' in df.columns:
        metrics_to_plot = expected_metrics + ['lr']
    else:
        metrics_to_plot = expected_metrics

    for metric_name in expected_metrics:
        if metric_name not in df.columns: #or f'val_{metric_name}' not in df.columns:
            print(f"Upozorenje: Kolone '{metric_name}' ili 'val_{metric_name}' nisu pronađene. Preskačem plot za '{metric_name}'.")
            continue

        plt.figure(figsize=(10, 6))
        sns.lineplot(data=df, x=df.index, y=metric_name, label=f'Trening {metric_name.replace("_", " ").title()}')
        #sns.lineplot(data=df, x=df.index, y=f'val_{metric_name}', label=f'Validacija {metric_name.replace("_", " ").title()}')
        plt.title(f'{metric_name.replace("_", " ").title()} tokom Treninga')
        plt.xlabel('Epoha')
        plt.ylabel(metric_name.replace("_", " ").title())
        plt.legend()
        plt.grid(True)
        
        # Kreiraj direktorijum za izlazne plotove ako ne postoji
        os.makedirs(output_dir, exist_ok=True)
        plot_path = os.path.join(output_dir, f'{metric_name}_plot.png')
        plt.savefig(plot_path)
        print(f"Sačuvan plot za '{metric_name}' na: {plot_path}")
        plt.close() # Zatvori plot da ne zauzima memoriju

    # Poseban plot za Learning Rate ako postoji
    if 'lr' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=df, x=df.index, y='lr', label='Learning Rate')
        plt.title('Learning Rate tokom Treninga')
        plt.xlabel('Epoha')
        plt.ylabel('Learning Rate')
        plt.grid(True)
        plot_path = os.path.join(output_dir, 'learning_rate_plot.png')
        plt.savefig(plot_path)
        print(f"Sačuvan plot za 'lr' na: {plot_path}")
        plt.close()

    print("\nVizualizacija metrika završena.")

In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

def plot_training_metrics_subplots(csv_file_path: str, output_path: str = 'training_summary_plots.png'):
    """
    Učitava metrike treninga i validacije iz CSV fajla i prikazuje ih kao subplotove
    na jednoj velikoj figuri.
    """
    if not os.path.exists(csv_file_path):
        print(f"Greška: Fajl '{csv_file_path}' ne postoji.")
        return

    try:
        df = pd.read_csv(csv_file_path)
    except Exception as e:
        print(f"Greška pri čitanju CSV fajla '{csv_file_path}': {e}")
        return

    expected_metrics = [
        'loss', 'accuracy', 'f1_score', 'roc_auc', 'average_precision'
    ]
    
    existing_metrics = []
    
    for metric in expected_metrics:
        if metric in df.columns:
            # Dodaj samo metrike koje imaju trening vrednosti (validacione će biti automatski dodate)
            existing_metrics.append(metric)

    if 'lr' in df.columns:
        existing_metrics.append('lr')

    num_plots = len(existing_metrics)
    if num_plots == 0:
        print("Upozorenje: Nisu pronađene validne kolone metrika za plotovanje.")
        return

    cols = math.ceil(math.sqrt(num_plots))
    rows = math.ceil(num_plots / cols)

    plt.figure(figsize=(cols * 6, rows * 4)) 
    
    # 4. Prolazak kroz metrike i plotovanje
    for i, metric_name in enumerate(existing_metrics):
        
        ax = plt.subplot(rows, cols, i + 1)
        
        title_name = metric_name.replace("_", " ").title()
        
        # Plot za Learning Rate
        if metric_name == 'lr':
            sns.lineplot(data=df, x=df.index, y='lr', label='Learning Rate', ax=ax)
            ax.set_title(f'Stopa Učenja (Learning Rate)')
            ax.set_ylabel('Learning Rate')
        
        # Plot za Trening/Validaciju metrike
        else:
            
            sns.lineplot(data=df, x=df.index, y=metric_name, label=f'Trening {title_name}', ax=ax)
            
            
            val_col = f'val_{metric_name}'
            if val_col in df.columns:
                sns.lineplot(data=df, x=df.index, y=val_col, label=f'Validacija {title_name}', ax=ax)
            
            ax.set_title(f'{title_name} tokom Treninga')
            ax.set_ylabel(title_name)

        # Postavljanje zajedničkih elemenata za sve subplotove
        ax.set_xlabel('Epoha')
        ax.legend()
        ax.grid(True)

    plt.tight_layout() # Automatski podešava subplotove da se ne preklapaju
    
    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    plt.savefig(output_path)
    print(f"\nSačuvan sveobuhvatni grafikon metrika na: {output_path}")
    plt.close()


In [6]:
csv_file = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/final_results_graphs_only/gcn/gcn_history-max-e50.csv' 
    
output_directory_for_plots = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gcn' 

plot_training_metrics_subplots(csv_file, output_directory_for_plots)


Sačuvan sveobuhvatni grafikon metrika na: /home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gcn


In [7]:
csv_file = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/final_results_graphs_only/gSage/gSage_model-e50.csv' 
    
output_directory_for_plots = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gSage' 

plot_training_metrics_subplots(csv_file, output_directory_for_plots)


Sačuvan sveobuhvatni grafikon metrika na: /home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gSage


In [8]:
csv_file = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/final_results_graphs_only/gin/gin_history-e50.csv' 
    
output_directory_for_plots = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gin' 

plot_training_metrics_subplots(csv_file, output_directory_for_plots)


Sačuvan sveobuhvatni grafikon metrika na: /home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots_graphs_only/gin


In [9]:
csv_file = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/final_results/gat/gat_history_e50.csv' 
    
output_directory_for_plots = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots+g+f/gat' 

plot_training_metrics_subplots(csv_file, output_directory_for_plots)


Sačuvan sveobuhvatni grafikon metrika na: /home/ivana-milutinovic/Documents/Doktorske/BIORad/GitHub/BIO-info-multiclass/output_plots+g+f/gat


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_optuna_res(csv_path: str, save_directory: str, plot_name:str):
    
    METRIKE = ['loss', 'f1_score', 'val_loss', 'val_f1_score'] 
    
    try:
        # Učitavanje podataka
        df = pd.read_csv(csv_path)
        
        # Provera da li su sve potrebne kolone prisutne
        missing_cols = [col for col in METRIKE if col not in df.columns]
        if missing_cols:
            print(f"Greška: Sledeće kolone nedostaju u fajlu '{csv_path}': {missing_cols}")
            print(f"Dostupne kolone su: {df.columns.tolist()}")
            return # Prekida izvršavanje funkcije
        
        # Kreiranje figure i subplota
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
        fig.suptitle('Metrike treniranja modela po epohama', fontsize=16)
        
        # Flatten 2x2 matricu subplota u listu radi lakše iteracije
        axes = axes.flatten() 
        
        for i, metric in enumerate(METRIKE):
            # Određivanje naslova i boje
            if 'loss' in metric:
                title_prefix = 'Gubitak (Loss)'
                boja = 'tab:red' if 'val' not in metric else 'tab:orange'
            elif 'f1_score' in metric:
                title_prefix = 'F1-mera (F1-Score)'
                boja = 'tab:blue' if 'val' not in metric else 'tab:green'
            
            set_type = 'Validacija' if 'val' in metric else 'Trening'
            
            # Crtanje linijskog grafa (x-osa su indeksi = epohe)
            axes[i].plot(df.index, df[metric], label=metric, color=boja)
            
            # Postavljanje naslova i labele
            axes[i].set_title(f"{title_prefix} na {set_type} setu")
            axes[i].set_xlabel('Epoha')
            axes[i].set_ylabel(metric.capitalize())
            axes[i].grid(True, linestyle='--', alpha=0.6)
            axes[i].legend()

        # Automatsko prilagođavanje prostora između subplota
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        # --- ČUVANJE SLIKE ---
        
        # Kreiranje direktorijuma ako ne postoji
        os.makedirs(save_directory, exist_ok=True)
        
        # Definicija putanje za čuvanje fajla
        file_name = f'{plot_name}.png'
        save_path = os.path.join(save_directory, file_name)
        
        # Čuvanje
        plt.savefig(save_path)
        plt.close(fig) # Zatvaranje figure da se oslobodi memorija
        
        print(f"Uspešno sačuvano: {save_path}")

    except FileNotFoundError:
        print(f"Greška: CSV fajl nije pronađen na putanji: {csv_path}")
    except pd.errors.EmptyDataError:
        print(f"Greška: CSV fajl je prazan: {csv_path}")
    except Exception as e:
        print(f"Došlo je do neočekivane greške: {e}")

In [8]:
optuna_csv_gat = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/bio-inf-all_data/optuna_csvs/history_results_11.csv'
output_dir_plots = '/home/ivana-milutinovic/Documents/Doktorske/BIORad/bio-inf-all_data/optuna_csvs/plots'
plot_optuna_res(optuna_csv_gat, output_dir_plots, 'optuna_plots')

Uspešno sačuvano: /home/ivana-milutinovic/Documents/Doktorske/BIORad/bio-inf-all_data/optuna_csvs/plots/optuna_plots.png
